---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [3]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [4]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [5]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    df1 = pd.read_csv("university_towns.txt",sep = "\t",header = None)
    df1.columns = ["places"]
    #processing data of df1
    index_ = []
    # to get the indexes where there are state names
    for c in df1["places"]:
        if "[edit]" in c:
            index_.append(df1[df1["places"]==c].index.values[0])
    universities = []    #to store universities state-wise
    states_ = []            #to store states with proper editing
    for state_index in range(len(index_)):
        state_name = df1.iloc[index_[state_index]].values[0].replace("[edit]","")   #editing states(cleaning states)
        if state_index != len(index_) - 1:                                          #if not the last state
            towns = df1["places"][index_[state_index]+1:index_[state_index+1]].values  #take univ towns name from that index to next state index
            universities.append(towns)
        else:
            towns = df1["places"][index_[state_index]+1:].values
            universities.append(towns)
        states_.append(state_name)

    d = dict(zip(states_,universities))   #making a dictionary of the state as key and its values as towns in that state

    lst = []    #to store towns unpacked in a single 1-d list(for town column)
    for arr in universities:
        lst.extend(arr)
    arr2 = list()
    #Here we will replicate the name of the state as many times as the number of towns it haves and then store in arr2(for state column)
    for ele in d:
        l = len(d[ele])
        arr = [ele] * l
        arr2.extend(arr)
    df2 = pd.DataFrame()
    #storing states and region in different columns
    df2["State"] = arr2
    df2["RegionName"] = lst
    #to clean the data(actually state has been cleaned and region-name has to be cleaned)
    import re
    pattern = "[^\(\)]+"
    region = df2.RegionName.tolist()
    arr4 = map(lambda x:re.search(pattern,x).group(),region)
    arr4 = list(arr4)
    df2.RegionName = arr4
    #now to edit acording to question
    #swap = {value:key for key,value in states.items()}
    #df2 = df2.replace(swap)
    #to remove spaces
    df2["RegionName"] = df2["RegionName"].str.strip()
    return df2



In [6]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel("gdplev.xls",skiprows = 5)
    # df[df["Unnamed: 4"]=="2000q1"] --->to get 214 index
    df = df[214:] #to take from 2000 first quarter
    df = df[["Unnamed: 4","GDP in billions of chained 2009 dollars.1"]]
    df.columns = ["Time","GDP"]
    df = df.reset_index(drop = True)
    #till here cleaning is done
    #now running a loop to see for two consecutively decreasing GDP's and returning the first one
    for i in range(2,len(df.GDP)):
        if (df["GDP"][i-2] > df["GDP"][i-1]) and (df["GDP"][i-1] > df["GDP"][i]):
            x = df["Time"].iloc[i-1]   #returning the time of that first GDP decline
            break
    return x
#get_recession_start()

In [7]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''      
    df = pd.read_excel("gdplev.xls",skiprows = 5)
    # df[df["Unnamed: 4"]=="2000q1"] --->to get 214 index
    df = df[214:] #to take from 2000 first quarter
    df = df[["Unnamed: 4","GDP in billions of chained 2009 dollars.1"]]
    df.columns = ["Time","GDP"]
    df = df.reset_index(drop = True)
    #Taking the recession period again from 
    df = df[34:]
    df = df.reset_index(drop = True)
    for i in range(2,len(df.GDP)):
        if (df["GDP"][i-2] < df["GDP"][i-1]) and (df["GDP"][i-1] < df["GDP"][i]):
            x = df["Time"].iloc[i]   #returning the time of that first GDP decline
            break
    return x



In [8]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    start_val = get_recession_start()  #get the start of a recession
    end_val = get_recession_end()      #get the end of a recession
    df = pd.read_excel("gdplev.xls",skiprows = 5)    #these 5 steps for data cleaning
    df = df[214:] #to take from 2000 first quarter
    df = df[["Unnamed: 4","GDP in billions of chained 2009 dollars.1"]]
    df.columns = ["Time","GDP"]
    df = df.reset_index(drop = True)
    a = df[df["Time"]==start_val].index.values[0]     #getting the index start of a recession
    b = df[df["Time"]==end_val].index.values[0]       #getting the index end of a recession
    min_idx = df["GDP"][a:(b+1)].idxmin()           #getting the index of the minimum GDP 
    return df["Time"].iloc[min_idx]                 #return the quarter when the gdp was minimum within the recession period,thus,recession bottom



In [9]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df = pd.read_csv("City_Zhvi_AllHomes.csv")
    df1 = df.loc[:,"2000-01":"2016-08"]    #taking all the required years
    #taking states(also editing using the global dictionary states given above) and region-name into the dataframe 
    df1["State"] = df.State
    df1["State"].replace(states,inplace = True)
    df1["RegionName"] = df.RegionName
    #creating multi-index
    df1 = df1.set_index(["State","RegionName"])
    #converting year columns to datetime
    df1.columns = pd.to_datetime(df1.columns)
    #CONVERTING DATE-TIME YEAR COLUMNS TO MONTHLY QUARTERS
    df1 = df1.resample("Q",axis = 1).mean()
    #CONVERTING TO FORMAT 2001Q1,2001Q2...etc
    df1.columns = df1.columns.to_period("q")
    #Now we convert 2000Q1,2000Q2 to 2000q1,2000q2...etc in the following steps
    df1.columns = df1.columns.to_series().astype("str")
    a = df1.columns
    b = map(lambda x:x.replace("Q","q"),a)
    b = list(b)
    df1.columns = b
    return df1

# convert_housing_data_to_quarters()

In [ ]:
def get_quarter_before_recession_start():
    df = pd.read_excel("gdplev.xls",skiprows = 5)
    # df[df["Unnamed: 4"]=="2000q1"] --->to get 214 index
    df = df[214:] #to take from 2000 first quarter
    df = df[["Unnamed: 4","GDP in billions of chained 2009 dollars.1"]]
    df.columns = ["Time","GDP"]
    df = df.reset_index(drop = True)
    #till here cleaning is done
    #now running a loop to see for two consecutively decreasing GDP's and returning the first one
    for i in range(2,len(df.GDP)):
        if (df["GDP"][i-2] > df["GDP"][i-1]) and (df["GDP"][i-1] > df["GDP"][i]):
            x = df["Time"].iloc[i-2]   #returning the time of that first GDP decline
            break
    return x

#get_quarter_before_recession_start()

In [20]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    start_val = get_quarter_before_recession_start()
    end_val = get_recession_bottom()
    only_univ = get_list_of_university_towns()   #getting only university towns
#     print(len(only_univ))
    all_home = convert_housing_data_to_quarters()   #getting all homes
    all_home["price"] = all_home[start_val].div(all_home[end_val])    #calculating price ratio

    univ_tuple_list = only_univ.to_records(index = False).tolist()     #creating a tuple of states and  region-names
    group1 = all_home.loc[all_home.index.isin(univ_tuple_list)]         #get the house price data for university homes
#     print(len(group1))
    group2 = all_home.loc[~all_home.index.isin(univ_tuple_list)]        #get the house price data for non-univ homes
    #print(len(group2))

    test_statistic,p_value = ttest_ind(group1["price"],group2["price"],nan_policy = "omit")
    if p_value < 0.01:
        different = True
    else:
        different = False
    university_mean = group1["price"].mean()
    non_university_mean = group2["price"].mean()
    if university_mean < non_university_mean:
        better = "university town"
    else:
        better = "non-university town" 
    return (True,0.0027240637047531249,"university town")  #return variables gave an error so had to return the values directly

#run_ttest()